In [ ]:
import os
import pydicom
from PIL import Image

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pydicom
import cv2
import SimpleITK as sitk
from PIL import Image
 
from skimage.morphology import disk, rectangle, binary_dilation, binary_erosion, binary_closing, binary_opening
 
def read_dicom_data(file_name):
    file = sitk.ReadImage(file_name)
    data = sitk.GetArrayFromImage(file)
    data = np.squeeze(data, axis=0)
    data = np.int32(data)
    dicom_dataset = pydicom.dcmread(file_name,force=True)
    # slice_location = dicom_dataset.SliceLocation         #获取层间距
    return data, data.shape[0], data.shape[1]
 
def window(window, img_data):
    if window == 'Lung':
        img_data[img_data < -1150] = -1150
        img_data[img_data > 350] = 350
    elif window == 'Pelvic':
        img_data[img_data < -138] = -138
        img_data[img_data > 238] = 238
    elif window == 'Chest':
        img_data[img_data < -160] = -160
        img_data[img_data > 240] = 240
    elif window == 'Chest_scatter':
        img_data[img_data < -752] = -752
        img_data[img_data > 838] = 838
    elif window == 'Pelvic_scatter':
        img_data[img_data < -300] = -300
        img_data[img_data > 240] = 240
    else:
        img_data[img_data < 0] = 0
        img_data[img_data > 80] = 80
    return img_data
 
def find_max_region(mask_sel):
    contours, hierarchy = cv2.findContours(mask_sel, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # 找到最大区域并填充
    area = []
    for j in range(len(contours)):
        area.append(cv2.contourArea(contours[j]))
    max_idx = np.argmax(area)
    max_area = cv2.contourArea(contours[max_idx])
    for k in range(len(contours)):
        if k != max_idx:
            cv2.fillPoly(mask_sel, [contours[k]], 0)
    return mask_sel
 
def quchuang(dcm_path,window1,window2):
    # 读入dicom图像
    dcm_path = dcm_path
    pixel_array, rows, columns = read_dicom_data(dcm_path)
    pixel_array1 = pixel_array.copy()
    pixel_array2 = pixel_array.copy()
 
    pixel_array1 = window(window1, pixel_array1)
    imageData1 = (pixel_array1 - pixel_array1.min()) * 255.0 / (pixel_array1.max() - pixel_array1.min())
    imageData1 = np.uint8(imageData1)
 
    pixel_array2 = window(window2, pixel_array2)
    imageData2 = (pixel_array2 - pixel_array2.min()) * 255.0 / (pixel_array2.max() - pixel_array2.min())
    imageData2 = np.uint8(imageData2)
 
    # 二值化
    ret, binary = cv2.threshold(imageData2, 3, 255, cv2.THRESH_BINARY)
 
    # 腐蚀
    selem = disk(3)
    fushi = binary_erosion(binary, selem)
 
    # 找最大连通区域
    binary = np.uint8(fushi)
    max_region = find_max_region(binary)
 
    # 膨胀
    selem = disk(3)
    pengzhang = binary_dilation(max_region, selem)
    
    #最左最右边两列像素值为0
    pengzhang[:, 0] = 0
    pengzhang[:, 511] = 0
    
    #填充
    A = np.uint8(pengzhang)
    h, w = A.shape[:2]
    #print(pengzhang.shape[:2])
    mask_tp = np.zeros((h + 2, w + 2), np.uint8)
    temp = A.copy()
    temp2 = A.copy()
    cv2.floodFill(temp, mask_tp, (1, 1), 255)
    cv2.floodFill(temp2, mask_tp, (w - 2, h - 2), 255)
    rt = cv2.bitwise_not(temp)
 
    Tianchong = rt
    Tianchong[rt > 0] = 255
 
    image_process = np.uint8((Tianchong / 255) * imageData1)
    return image_process


In [ ]:
from tqdm import tqdm
path="/disk8t/jialiangfan/trained_models/dataset/medical_data/"
for root, dirs, files in os.walk(path):
    for file in tqdm(files):
        if file.endswith('.DCM') or file.endswith('.dcm'):
            dcm_path = os.path.join(root, file)
            window1 = 'ss'
            window2 = 'ss'
            try:
                imagxe_array = quchuang(dcm_path, window1, window2)
                #image = Image.fromarray(image_array)
                # print(os.path.relpath(root,file),'ww')
                # print(root,'this is root')
                # print(dirs,'this is folder_path')
                # break
                save_dir = os.path.join('/disk8t/jialiangfan/trained_models/dataset/medical_data/medical_images/', os.path.relpath(root, path))
                os.makedirs(save_dir, exist_ok=True)
                if file.endswith('.DCM'):
                    save_path = os.path.join(save_dir, file.replace('.DCM', '.jpg'))
                elif file.endswith('.dcm'):
                    save_path = os.path.join(save_dir, file.replace('.dcm', '.jpg'))
                cv2.imwrite(save_path, imagxe_array)
                print(f"Converted {dcm_path} to {save_path}")
            except (ValueError, RuntimeError) as e:
                print(f"Error converting {dcm_path}: {e}")
                continue